**version 5.3**

# Input basics

This example shows the basics of input generation using the low level IO package `ascot5io`.

1. <a href='#bfield'>Magnetic and electric field</a>
2. <a href='#markers'>Markers</a>
3. <a href='#options'>Options</a>
4. <a href='#wall'>Wall</a>
5. <a href='#other'>Other inputs</a>

### Necessary imports

In [1]:
import numpy as np
from a5py.ascot5io.ascot5 import Ascot

<a id='bfield'></a>

## Magnetic and electric field

Having valid magnetic and electric field inputs is necessary for all ASCOT5 simulations. There are several different magnetic field input types to choose from, but the one most commonly used is `B_2DS` which represents axisymmetric tokamak field interpolated using bicubic splines.

In `B_2DS` module, the magnetic field poloidal component is evaluated from the poloidal flux as

$$B_R = -\frac{1}{R}\frac{\partial\psi}{\partial z} \qquad B_z = \frac{1}{R}\frac{\partial\psi}{\partial R}$$

and the toroidal component is given as an input. The poloidal flux is in addition used to evaluate the normalized poloidal flux which is used as a flux coordinate. 

**Note**
It is possible to give also the poloidal component separately, but in this case it is summed to the components calculated from the poloidal flux so this possibility is rarely used. If one insists on giving the poloidal components explicitly, one can avoid double counting by multiplying the poloidal flux with a small number but be cautious here as something might break.

There are higher level tools that read EQDSK and write it as a `B_2DS` input (see `a5py/preprocessing/eqdsk2input.py`) but here the point is to demonstrate the lower level tools. Before demonstration, we need to create suitable data using the analytical model (see `B_GS` if you wish to learn the details):

In [ ]:
import a5py.ascot5io.B_GS as B_GS

# ITER-like but circular equilibrium
psi_mult  = 200
R0        = 6.2
z0        = 0
Bphi0     = 5.3
psi_coeff = np.array([ 2.218e-02, -1.288e-01, -4.177e-02, -6.227e-02,
                       6.200e-03, -1.205e-03, -3.701e-05,  0,
                       0,          0,          0,          0,         -0.155])

Rmin = 4; Rmax = 8.5; nR = 120; zmin = -4; zmax = 4; nz = 200;
B_GS.write_hdf5_B_2DS("input_basics.h5", R0, z0, Bphi0, psi_mult,
                      psi_coeff, Rmin, Rmax, nR, zmin, zmax, nz, desc="Data")

These commands construct an ITER-like magnetic field using the analytical model. The analytical model is then evaluated at the grid points and written as `B_2DS` field in the example file "input.basics.h5".

Now to demonstrate what data `B_2DS` actually requires, we read the data we just wrote and rewrite it.

In [3]:
import a5py.ascot5io.B_2DS as B_2DS

h5 = Ascot("input_basics.h5")
data = h5.bfield["Data"].read()

B_2DS.write_hdf5(
    "input_basics.h5",                                   # Filename
    axisr=data["axisr"], axisz=data["axisz"],            # Magnetic axis R,z coordinates
    psi0=data["psi0"], psi1=data["psi1"],                # Psi value at the axis and at the separatrix
    nr=data["nr"], rmin=data["rmin"], rmax=data["rmax"], # R coordinates which define the R,z grid
    nz=data["nz"], zmin=data["zmin"], zmax=data["zmax"], # z coordinates which define the R,z grid
    psi=data["psi"],                                     # Tabulated poloidal flux at the R,z grid points
    bphi=data["bphi"],                                   # Tabulated toroidal field at the R,z grid points
    br=data["br"], bz=data["bz"],                        # Poloidal field components but these are usually set to zero
    desc="Samedata"                                      # Description / name of the data
)

'B_2DS_3036893075'

There are couple of things that are worth pointing out. There is not a general rule how dense the R,z grid should be. One measure of the field quality is (numerical) divergence. However, the 2D field is guaranteed to be divergence free in the absence of explicitly given $B_\mathrm{pol}$ ($\partial_\phi B_\phi = 0$ and $B_\mathrm{pol}$ is calculated from $\psi$). Other way to validate the grid is to vary the grid resolution and simulate some particles collisionlessly to see if there is artificial transport.

A common issue is that some particles near the axis are aborted. If this happens, it is probably because the square root of the normalized poloidal flux, $\rho$, has become imaginary. This can be fixed by adding some padding to `psi0` value.

It is common in cases where ASCOT5 is used that the electric field plays no important part. The easy way then to include the electric field is to use the `E_TC` input (Trivial Cartesian electric field), which defines an electric field with constant xyz components, and set all components to zero:

In [4]:
import a5py.ascot5io.E_TC as E_TC

E_TC.write_hdf5("input_basics", exyz=np.array([0, 0, 0]), desc="ZeroEfield")

'E_TC_2706624175'